In [1]:
import pandas as pd
import os, csv
import numpy as np
from sqlalchemy import create_engine


In [3]:
data = os.path.join('Resources','cleaned_merged','cleaned_netflix_sag_data.csv')
data1 = pd.read_csv(data)
data_df = pd.DataFrame(data1).drop(columns='Unnamed: 0')
data_df

,id,show_id,type,director,full_name,release_year,description,category,won
0,279,80201906,Movie,Ryan Coogler,Chadwick Boseman,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
1,280,80201906,Movie,Ryan Coogler,Michael B. Jordan,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
2,282,80201906,Movie,Ryan Coogler,Danai Gurira,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
3,283,80201906,Movie,Ryan Coogler,Martin Freeman,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
4,284,80201906,Movie,Ryan Coogler,Daniel Kaluuya,2018,"T'Challa, the superpowered new leader of the h...",CAST IN A MOTION PICTURE,True
...,...,...,...,...,...,...,...,...,...
274,28777,607931,Movie,Jocelyn Moorhouse,Kate Nelligan,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
275,28778,607931,Movie,Jocelyn Moorhouse,Alfre Woodard,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
276,28784,607931,Movie,Jocelyn Moorhouse,Maya Angelou,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
277,28788,607931,Movie,Jocelyn Moorhouse,Jean Simmons,1995,A conflicted young woman spends the summer wit...,CAST IN A MOTION PICTURE,False
